In [1]:
from src.datasets import load_celeba
from src.trainer import Trainer

In [2]:
import torch

In [3]:
X_train_sampler, X_test_sampler, Y_train_sampler, Y_test_sampler = load_celeba(
    img_size=64,
    batch_size=64
)

Files already downloaded and verified


In [4]:
trainer = Trainer(
    img_size=64,
    batch_size=64,
    zc=1,
    z_std=0.1,
    z_size=8,
    T_iters=10,
    D_lr=1e-4,
    T_lr=1e-4,
    gamma_0=0,
    gamma_1=2/3,
    gamma_iters=25000
)

In [5]:
trainer.train(
    X_train_sampler,
    X_test_sampler,
    Y_train_sampler,
    Y_test_sampler
)

  0%|          | 0/100001 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 768.00 MiB (GPU 0; 7.79 GiB total capacity; 5.59 GiB already allocated; 199.06 MiB free; 6.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF